In [1]:
import duckdb

In [ ]:
# Connect to (or create) a DuckDB file
conn = duckdb.connect('../data/raw.duckdb')

# Save your clean DataFrame as a DuckDB table
conn.execute("CREATE TABLE IF NOT EXISTS elasticity AS SELECT * FROM train_df2")

In [ ]:
df = conn.execute("SELECT * FROM elasticity").fetchdf()
